In this notebook we present comparison of [learning curves](https://en.wikipedia.org/wiki/Learning_curve_(machine_learning)) of best performing models trained in notebook `02. Training ML models`.

This comparison justifies that using more data wont further improve the estimation errors.

In [1]:
#loading dataset
import numpy as np

input_and_output = np.load("../final/dataset.npz")
inputs  = input_and_output["inputs"].astype(np.float64)
inputs  = input_and_output["inputs"].astype(np.float64)
outputs = input_and_output["outputs"].astype(np.float64)
dataset_size = inputs.shape[0]

print("loaded dataset")

# transforming time profiles to its orders of magnitude

LOWER_LIMIT = -9

def output_transform(outputs: np.array) -> np.array:
    x = np.copy(outputs)
    zeros_in_output = x <= 0
    x[zeros_in_output] = 1
    y = np.log10(x)
    y[zeros_in_output] = LOWER_LIMIT
    y[y < LOWER_LIMIT] = LOWER_LIMIT
    return y
    
def output_untransform(transformed_outputs: np.array) -> np.array:
    lower_limits = transformed_outputs <= LOWER_LIMIT
    z = 10 ** transformed_outputs
    z[lower_limits] = 0
    return z

outputs_order_of_magnitude = output_transform(outputs)
print("transformed to orders of magnitude")

# dropping treatment column in input

def drop_treatment(input_data: np.ndarray) -> np.ndarray:
    """Drops treatment data from the dataset"""
    if input_data.shape[1] == 11:
        return input_data[:, 1:]

    return input_data

input_without_treatment = drop_treatment(inputs)

print("dropped treatment column")

#splitting data into train, test, validate datasets 
train_size = int(dataset_size * 0.7)
test_size = int(dataset_size * 0.15)

X_train = input_without_treatment[:train_size, :]
Y_train = outputs_order_of_magnitude[:train_size, :]
print(f"train sizes: {X_train.shape}, {Y_train.shape}")
X_test = input_without_treatment[train_size:(train_size + test_size), :]
Y_test = outputs_order_of_magnitude[train_size:(train_size + test_size), :]
print(f"test sizes: {X_test.shape}, {Y_test.shape}")

print("train test split")

# scaling inputs

import pickle
from pathlib import Path

LOGNORMAL_PARAMETERS = (1, 2)

class CustomScaler:
    def __init__(self):
        super().__init__()
        self.scaler = MinMaxScaler()
        self.plot_loval = [0.0] * len(LOGNORMAL_PARAMETERS)
        self.plot_hival = [1.0] * len(LOGNORMAL_PARAMETERS)

    def transform(self, x: np.ndarray, copy=None) -> np.ndarray:
        res = self.scaler.transform(x)
        for i, parameter_index in enumerate(LOGNORMAL_PARAMETERS):
            res[:, parameter_index] = (x[:, parameter_index] - self.plot_loval[i]) / (self.plot_hival[i] - self.plot_loval[i])

        return res

    def fit(self, x, copy=None):
        self.scaler.fit(x)
        for i, parameter_index in enumerate(LOGNORMAL_PARAMETERS):
            column_values = x[:, parameter_index]

            quantile_1, quantile_3 = np.quantile(column_values, [0.25, 0.75], axis=0)
            iqr = quantile_3 - quantile_1

            loval = quantile_1 - 1.5 * iqr
            hival = quantile_3 + 1.5 * iqr

            wiskhi = np.compress(column_values <= hival, column_values)
            wisklo = np.compress(column_values >= loval, column_values)
            actual_hival = np.max(wiskhi)
            actual_loval = np.min(wisklo)

            self.plot_loval[i] = actual_loval
            self.plot_hival[i] = actual_hival

        return self

    def inverse_transform(self, x, copy=None):
        res = self.scaler.inverse_transform(x)
        for i, parameter_index in enumerate(LOGNORMAL_PARAMETERS):
            res[:, parameter_index] = x[:, parameter_index] * (self.plot_hival[i] - self.plot_loval[i]) + self.plot_loval[i]
        return res

with Path(f"../final/scaler.pickle").open("rb") as scaler_file:
    scaler = pickle.load(scaler_file)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("scaled")

# applying principal component analysis

from sklearn.decomposition import PCA
PCA_COMPONENTS=12
with Path(f"../final/pca{PCA_COMPONENTS}.pickle").open("rb") as opened_file:
    pca = pickle.load(opened_file)
Y_train_pca = pca.transform(Y_train)

print("applied pca")

import time
from sklearn.metrics import mean_squared_error
import optuna

loaded dataset
transformed to orders of magnitude
dropped treatment column
train sizes: (700000, 10), (700000, 200)
test sizes: (150000, 10), (150000, 200)
train test split
scaled
applied pca


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNet, Ridge, LinearRegression, LassoLars
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, BaggingRegressor, HistGradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor

for study_name, model_builder in [
    ("MLPRegressor_constant_600", lambda t: MLPRegressor(**t.user_attrs["model_params"]))
    ("LassoLars", lambda t: None),
    "LinearRegression",
    "Ridge",
    "ElasticNet",
    "HistGradientBoostingRegressor",
    "BaggingRegressor",
    "RandomForrest",
    "ExtraTreesRegressor",
    "KNeighborsRegressor"
]:
    pass